# Goals
- train predictor on training trusted and untrusted sets 
- use confidence as tampering score

In [7]:
from cupbearer import data, detectors, models, scripts, tasks, utils
from torch.utils.data import DataLoader

# Train Measurement Predictor

In [4]:
train_data = data.TamperingDataset("diamonds", train=True)
val_data = data.TamperingDataset("diamonds", train=False)

In [6]:
transformer, tokenizer, emb_dim, max_len = models.transformers_hf.load_transformer(
    "pythia-14m"
)
model = models.TamperingPredictionTransformer(
        model=transformer,
        tokenizer=tokenizer,
        embed_dim=emb_dim,
        max_length=max_len,
        n_sensors=3,
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
scripts.train_classifier(
    path=(classifier_path := utils.log_path("logs/tampering/predictor")),
    model=model,
    train_loader=DataLoader(train_data, batch_size=1, shuffle=True),
    task="multilabel",
    num_labels=4,
    val_loaders=DataLoader(val_data, batch_size=1, shuffle=False),
    max_epochs=3
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type                           | Params
------------------------------------------------------------------
0 | model          | TamperingPredictionTransformer | 7.6 M 
1 | train_accuracy | MultilabelAccuracy             | 0     
2 | val_accuracy   | ModuleList                     | 0     
3 | test_accuracy  | ModuleList                     | 0     
------------------------------------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.517    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   1%|          | 136/25000 [00:19<1:00:09,  6.89it/s, train/loss=1.510]   

/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


{'train/loss': tensor(1.4018, device='mps:0'),
 'train/acc_step': tensor(0.7500, device='mps:0')}